In [ ]:
from tkinter import Tk, Label, Button, Text, filedialog, Menu, Frame
import tkinter as tk
import pyaudio
import wave
import os
import re
import sys, traceback

def split_text(text):
    pattern1 = re.compile(r"[.]\n+|\n") #Tach doan
    pattern2 = re.compile(r"[.]+\s|\n") #Tach cau
    with open(text, "r", encoding='utf-8') as f:
        texts = f.read()
    #Tach doan nho 
    blocks = re.split(pattern1, texts)
    #Tach cau nho
    result = []
    for block in blocks:
        result += re.split(pattern2, block)
    result = [res for res in result if res != '']
    #In ra ket qua
    return result

# Mo text va hien thi
def openFile():
    label_count['text'] = '0'
    save_label['text'] = f"Saved {label_count['text']}.wav"
    save_label.pack_forget()
    file_name = filedialog.askopenfilename(initialdir = "./text",
                                                title = "Chon file .txt",
                                                filetypes = (("text files","*.txt"),("all files","*.*")))
    if file_name:
        text = split_text(file_name)
        total_sens['text'] = str(len(text))
        text_label['text'] = '@@'.join(text)
        lines = [str(i) + ".wav\n" + t for i, t in enumerate(text)]
        if not os.path.isdir(file_name[:-3]): os.mkdir(file_name[:-3])
        save_folder['text'] = file_name[:-3]
        with open(file_name[:-3]+"/Description.txt", "w", encoding='utf-8') as f:
            f.write('\n'.join(lines))
        sentence['text'] = label_count['text'] + ".wav\n\n" + text_label['text'].split('@@')[int(label_count['text'])]


#Chuong trinh ghi am
chunk = 1024  
sample_format = pyaudio.paInt16
channels = 2
fs = 44100
p = pyaudio.PyAudio()

end = False
def record():
    if label_count['text'] != total_sens['text'] and label_count['text'] != '-1':
        save_label.pack_forget()
        buttonStart.pack_forget()
        buttonNext.pack_forget()
        buttonStop.pack(pady=1,side = 'left', expand = True, fill = 'both')
        buttonNext.pack(pady=1,side = 'left', expand = True, fill = 'both')
        stream = p.open(format=sample_format,
                        channels=channels,
                        rate=fs,
                        frames_per_buffer=chunk,
                        input=True)
        frames = []
        def listen():
            global end
            if not end:
                data = stream.read(chunk)
                frames.append(data)
                window.after(1, listen)
            else:
                stream.stop_stream()
                filename = save_folder['text'] + "/" + label_count['text']
                wf = wave.open(filename+".wav", 'wb')
                wf.setnchannels(channels)
                wf.setsampwidth(p.get_sample_size(sample_format))
                wf.setframerate(fs)
                wf.writeframes(b''.join(frames))
                wf.close()
                end = False
        listen()
def next_sen():
    save_label.pack_forget()
    if label_count['text'] != total_sens['text']:
        label_count['text'] = str(int(label_count['text']) + 1)
        save_label['text'] = f"Saved {label_count['text']}.wav"
    if label_count['text'] == total_sens['text']:
        sentence['text'] = "Kết thúc!"
    else:
        sentence['text'] = label_count['text'] + ".wav\n\n" + text_label['text'].split('@@')[int(label_count['text'])]

def pre_sen():
    save_label.pack_forget()
    if label_count['text'] != '-1':
        label_count['text'] = str(int(label_count['text']) - 1)
        save_label['text'] = f"Saved {label_count['text']}.wav"
    if label_count['text'] == '-1':
        sentence['text'] = "Bắt đầu!"
    else:
        sentence['text'] = label_count['text'] + ".wav\n\n" + text_label['text'].split('@@')[int(label_count['text'])]

def stop():
    save_label.pack()
    print(save_label['text'])
    buttonStop.pack_forget()
    buttonNext.pack_forget()
    buttonStart.pack(pady=1,side = 'left', expand = True, fill = 'both')
    buttonNext.pack(pady=1,side = 'left', expand = True, fill = 'both')
    
    global end

    end = True
#Giao dien 
window = Tk()
window.title("Recorder")
window.geometry('400x400')
window.resizable(0, 0)
window.configure()
topframe = Frame(window)
topframe.pack(side="top", pady=1, anchor='center')
# Open button
buttonOpen = Button(topframe, text="Mở văn bản", font=("Times New Roman", 10), bg="grey", width=15, height=1, command=openFile)
buttonOpen.pack(side = 'left', expand = True, fill = 'both')
buttonOpen.pack(pady=1)


label_count = Label(text='0')
total_sens = Label(text='0')
save_folder = Label()
text_label = Label()
save_label = Label(window, text=f"Saved {label_count['text']}.wav")

sentence = Label(wraplength=350, anchor='w', justify='left', height=10, font=("Times New Roman", 18))
sentence.pack(side='top')

# Tao cac nut bam
bottomframe = Frame(window)
bottomframe.pack(side="bottom", pady=1, anchor='center')

# Pre button
buttonPre = Button(bottomframe, text="↶ Câu trước", font=("Times New Roman", 10), bg="orange", width=15, height=1, command=pre_sen)
buttonPre.pack(side = 'left', expand = True, fill = 'both')
buttonPre.pack(pady=1)

# Record button
buttonStart = Button(bottomframe, text="Bắt đầu ", font=("Times New Roman", 10), bg="green", width=15, height=1, command=record)
buttonStart.pack(side = 'left', expand = True, fill = 'both')
buttonStart.pack(pady=1)

# Next button
buttonNext = Button(bottomframe, text="Câu sau ↷", font=("Times New Roman", 10), bg="orange", width=15, height=1, command=next_sen)
buttonNext.pack(side = 'left', expand = True, fill = 'both')
buttonNext.pack(pady=1)

#Stop button
buttonStop = Button(bottomframe, text="Kết thúc", font=("Times New Roman", 10), bg="red", width=15, height=1, command=stop)#24b24e
window.mainloop()